In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.CenterCrop((64,960))])

In [23]:
import torchaudio

### Make noisy label dataset

In [24]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'ecaresnet50d_CQT_512_64'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_4840\4085875522.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4840\4085875522.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4840\4085875522.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4840\4085875522.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users

In [25]:
f = np.zeros((len(df_test_valid),19))
for fld in range(5):
    for kernels in [
        'ecaresnet50d_CQT_512_64'
                   ]:
        fname = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)
        fname = np.concatenate(fname)
         
        f += fname/5
        
f = pd.DataFrame(f)
f['ID'] = df_test_valid['song_id']
f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.281352,0.277935,0.018038,0.090174,0.005023,0.056496,0.005141,0.090486,0.085459,0.003146,0.080825,0.002025,0.000136,0.002204,0.000708,4.895144e-05,0.000096,0.000688,1.961059e-05,7072
1,0.006184,0.000571,0.000071,0.001078,0.000581,0.001000,0.000337,0.001195,0.000678,0.001620,0.000363,0.005106,0.979756,0.001326,0.000015,3.631644e-06,0.000005,0.000105,6.019178e-06,10207
2,0.109869,0.010498,0.003135,0.705925,0.000913,0.001297,0.030270,0.090234,0.027874,0.000056,0.017276,0.000013,0.001033,0.001116,0.000098,1.376609e-07,0.000391,0.000002,1.368468e-07,20008
3,0.577527,0.023951,0.003483,0.026586,0.017811,0.002459,0.221633,0.006364,0.014270,0.010443,0.086547,0.000242,0.006483,0.001731,0.000089,1.480402e-04,0.000011,0.000151,7.144268e-05,10924
4,0.139456,0.053832,0.065824,0.029821,0.636169,0.002822,0.025249,0.007497,0.004171,0.017252,0.007364,0.001320,0.000043,0.001534,0.000440,4.363915e-03,0.001823,0.000989,3.143751e-05,21896


In [26]:
consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)
consolNoisy.head()

C:\Users\User\AppData\Local\Temp\ipykernel_4840\4161917736.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.023273,0.381968,1.825197e-01,0.093455,2.919543e-02,9.258042e-02,2.101919e-03,0.055072,0.070133,0.009439,5.615117e-03,0.005179,0.000278,1.098200e-02,6.574837e-03,0.000784,1.771637e-02,1.310535e-02,2.902901e-05,10150
1,0.000005,0.000002,9.294359e-08,0.000002,3.281554e-09,3.224335e-07,4.803596e-10,0.000001,0.000003,0.999340,1.298669e-09,0.000001,0.000001,1.002681e-07,2.243995e-04,0.000419,2.076015e-10,1.434081e-07,1.518872e-08,11198
2,0.458789,0.029479,1.058781e-02,0.017429,2.424353e-02,2.636006e-03,4.397250e-02,0.003845,0.329465,0.005188,7.136635e-02,0.000350,0.000062,1.230593e-03,4.652597e-06,0.000475,8.763972e-06,2.510848e-04,6.168272e-04,13166
3,0.002213,0.000008,7.004205e-05,0.000019,9.960434e-01,9.498938e-07,3.127675e-04,0.000004,0.000236,0.000901,3.055594e-05,0.000008,0.000045,7.374911e-08,1.735766e-09,0.000101,1.530397e-07,3.543684e-10,5.255694e-06,12275
4,0.026710,0.041594,2.699048e-03,0.813383,2.819304e-04,2.509044e-02,3.370391e-03,0.040712,0.008593,0.002546,5.519975e-03,0.000438,0.000069,2.733119e-03,7.490209e-05,0.000005,2.578689e-02,2.229154e-04,1.694105e-04,24102


In [27]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df,
                 noisyDF = consolNoisy,
                 train = True,
                 valid = False,
                 transform=tfms,

                ):
        
        self.df = df
        self.train = train
        self.valid = valid
        self.transform = transform
        self.noisyDF = noisyDF
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        elif self.valid:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/train/{filename}')
        else:
            CQT = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/CQT/512_64/test/{filename}')
            
        return CQT


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            labels2 = self.noisyDF[self.noisyDF.ID==row.song_id].drop('ID',1).values[0]
            label = torch.tensor(row.genre_id,dtype=torch.long)
            label2 = torch.tensor(labels2).float()
            return samples, (label,label2)
        if self.valid:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:
            return samples

#### Loss

In [28]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=0.5):
        store_attr()
        self.focal_loss = CrossEntropyLossFlat()
        self.dice_loss =  BCEWithLogitsLossFlat()
        
    def __call__(self, pred, targ):
        if type(targ) is list:
            loss = self.focal_loss(pred, targ[0])*(1-self.alpha) + self.alpha * self.dice_loss(pred, targ[1])
        else:
            loss = self.focal_loss(pred, targ)
        return loss
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

#### FastAI model

In [29]:
import timm
# timm.create_model('efficientnet_b1', pretrained=False, in_chans=1)

In [30]:
NCLASS = df_train['genre_id'].nunique()
print(NCLASS)

19


In [31]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'ecaresnet50d_CQT_448_84_Noisy'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit =  CombinedLoss()#LabelSmoothingCrossEntropy(0.02) #CrossEntropyLossFlat() #FocalLoss() #CrossEntropyLossFlat() #FocalLoss() #LabelSmoothingCrossEntropy(0.01)
        batch_size = 64
        n_epochs = 10

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,train=False,valid=True)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('ecaresnet50d',in_chans=1,pretrained=False)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)
    
        f1_score = F1Score(average="micro")

        learn = Learner(dls, model,opt_func=Adam,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True,),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        learn.fit_one_cycle(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False,
                           valid=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.062233,2.534017,0.210447,03:16
1,0.975868,2.881295,0.275490,06:14
2,0.916699,2.701522,0.269965,07:05
3,0.863751,2.869370,0.247363,07:05
4,0.815572,2.099136,0.340784,07:06
5,0.769773,1.641189,0.468106,07:04
6,0.726768,1.621690,0.491713,05:26
7,0.678443,1.491216,0.523355,02:58
8,0.635159,1.351985,0.558011,02:58
9,0.606976,1.334566,0.559769,02:59


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.058433,4.374063,0.113762,02:58
1,0.974659,3.650954,0.206178,02:58
2,0.910693,1.831084,0.434204,02:58
3,0.856600,1.914018,0.417880,02:59
4,0.807418,1.545615,0.494726,02:58
5,0.766882,1.579131,0.491462,02:58
6,0.728964,1.618343,0.483425,02:58
7,0.683278,1.441548,0.526871,03:01
8,0.642728,1.357278,0.550728,02:59
9,0.615586,1.347069,0.556002,03:05


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.061262,3.901984,0.103968,03:05
1,0.981605,3.002738,0.183325,03:09
2,0.925908,2.152997,0.330738,03:02
3,0.867498,1.994970,0.345053,03:00
4,0.821018,1.832369,0.405826,03:00
5,0.777151,1.895262,0.392265,03:00
6,0.737253,1.660292,0.455048,03:01
7,0.690845,1.423455,0.517077,03:03
8,0.647360,1.373591,0.541436,03:03
9,0.621798,1.363522,0.544450,03:02


KeyboardInterrupt: 

In [ ]:
df_test_valid.head()

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### OOF

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'efficientnet_b2_CQT_512_64_Noisy'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

### Mode based sub

In [ ]:
kernel_type = 'efficientnet_b2_CQT_512_64_Noisy'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub: check OOF Score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop512_Mels128',
#                 'seresnet50t_MonoSpec_Hop448_Mels160',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

### Mode based sub

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_8bestmodels_CV_5862.csv',index=False)

In [ ]:
kernel_type

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
#                 'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_5bestmodels_CV_5877.csv',index=False)

### Avg probs

In [ ]:
# np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_0_probs.npy',allow_pickle=True)).shape,
df_test_valid.head()



In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_probEns_5bestmodels_NoCVInfo.csv',index=False)

#### Blind sub

In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/BlindSub_NoCVInfo.csv',index=False)

### End ###